**This notebook is copy of this one https://www.kaggle.com/steamedsheep/yolov5-is-all-you-need I just follow the advices from discussion and changed the input size from 3000 to 10000**

In [ ]:
import greatbarrierreef
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

# pytorch 

In [ ]:
!pip install '/kaggle/input/pytorch-190/torch-1.9.0+cu111-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-190/torchvision-0.10.0+cu111-cp37-cp37m-linux_x86_64.whl' --no-deps

# mmcv and mmdet 

In [ ]:

!cp -r /kaggle/input/mmdetectionv2140/* /kaggle/working/
!cp -r /kaggle/input/cbnet-custom/* /kaggle/working/
!pip install '/kaggle/working/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/working/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/working/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/working/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/working/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install '/kaggle/working/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

!pip install '/kaggle/input/timm-package/timm-0.4.12-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmcv1-3-18-cp37/mmcv_full-1.3.16-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install '/kaggle/input/mmcv1-3-18-cp37/icecream-2.1.1-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/needbao/executing-0.8.2-py2.py3-none-any.whl' --no-deps

In [ ]:
!mv /kaggle/working/CBNetV2-main /kaggle/working/CBNetV2
%cd /kaggle/working/CBNetV2
!pip install -e .


In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from tqdm import tqdm


import mmdet
import mmcv
# from mmdet.datasets import build_dataset
# from mmdet.models import build_detector
# from mmdet.apis import train_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed

In [ ]:
%cd ..

## Model


In [ ]:
#----------------yolo-----------------------------
# model = torch.hub.load('../input/yolov5-lib-ds', 
#                        'custom', 
#                        path='../input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt',
#                        source='local',
#                        force_reload=True)  # local repo
# model.conf = 0.01

In [ ]:
#--------------------cbnet----------------
config_file = '../input/pt-starfish/cbnet_starfish/cascade_mask_rcnn_cbv2_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_coco.py'
checkpoint_file= '../input/pt-starfish/cbnet_starfish/epoch_12_cs.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# infer

In [ ]:
#---------------------yolo---------------------------------
# for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
#     anno = ''
#     r = model(img, size=9000, augment=True)
#     if r.pandas().xyxy[0].shape[0] == 0:
#         anno = ''
#     else:
#         for idx, row in r.pandas().xyxy[0].iterrows():
#             if row.confidence > 0.3:
#                 anno += '{} {} {} {} {} '.format(row.confidence, int(row.xmin), int(row.ymin), int(row.xmax-row.xmin), int(row.ymax-row.ymin))
# #                 pred.append([row.confidence, row.xmin, row.ymin, row.xmax-row.xmin, row.ymax-row.ymin])
#     pred_df['annotations'] = anno.strip(' ')
#     env.predict(pred_df)

In [ ]:
#----------------------cbnet------------------------------
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    anno = ''
    rs = inference_detector(model, img)
    if len(rs[0])==0:
        anno = ''
        
    else:
        for r in rs[0]:
            print(r[-1])
            if r[-1]>0.3:
                anno += '{} {} {} {} {} '.format(r[-1], int(r[0]), int(r[1]), int(r[2]-r[0]), int(r[3]-r[1]))
    pred_df['annotations'] = anno.strip(' ')
    env.predict(pred_df)

In [ ]:
%cd /kaggle/working
!rm -rf ./mmpycocotools-12.0.3
!rm -rf ./mmcv-1.3.8
!rm -rf ./mmdetection-2.14.0
!rm -rf ./mmcv_full-1_3_8-cu110-torch1_7_0
!rm -rf ./CBNetV2
!rm -rf ./pycocotools-2.0.2
!rm -rf ./terminaltables-3.1.0-py3-none-any.whl
!rm -rf ./terminal-0.4.0-py3-none-any.whl
!rm -rf ./addict-2.4.0-py3-none-any.whl
!rm -rf ./yapf-0.31.0-py2.py3-none-any.whl
